## NLP Class Final Project

In [ ]:
!pip install pandarallel
!pip install gensim --upgrade
!pip install pyLDAvis --upgrade

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.8 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16674 sha256=1c914a492165182c287d0f6a1d86d90be9b20318b334feeb3c5b869c27098560
  Stored in directory: /root/.cache/pip/wheels/b9/c6/5a/829298789e94348b81af52ab42c19d49da007306bbcc983827
Successfully built pandarallel
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 60.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import nltk
import nltk.corpus
from nltk.text import Text
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from nltk.metrics import jaccard_distance
from nltk.text import Text
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from itertools import combinations
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import os
from pandarallel import pandarallel
import multiprocessing

import os
import warnings
warnings.simplefilter('ignore')

import gensim
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel

# import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)

nltk.download('all')

Mounted at /content/drive


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

True

In [ ]:
pandarallel.initialize(nb_workers = multiprocessing.cpu_count()-1, use_memory_fs=False, progress_bar = True)

INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [ ]:
%%time

df_news_final_project = pd.read_parquet('https://storage.googleapis.com/msca-bdp-data-open/news_final_project/news_final_project.parquet', engine='pyarrow')
df_news_final_project.shape

CPU times: user 7.78 s, sys: 5.92 s, total: 13.7 s
Wall time: 38.8 s


(199707, 5)

In [ ]:
df_news_final_project.head()

,url,date,language,title,text
0,http://www.dataweek.co.za/21690r,2024-04-05,en,Flash for AI - 28 March 2024 - EBV Electrolink - Dataweek,"\nFlash for AI - 28 March 2024 - EBV Electrolink - Dataweek\nHome\nAbout us\nBack issues / E-book / PDF\nSubscribe\nAdvertise\n\n \n\nEMP Handbook\n\nCategories\n\n▸ Editor's Choice\n▸ Multimedia, Videos\n▸ AI & ML\n▸ Analogue, Mixed Signal, LSI\n▸ Circuit & System Protection\n▸ Computer/Embedded Technology\n▸ Design Automation\n▸ DSP, Micros & Memory\n▸ Edge Computing & IIoT\n▸ Electronics Technology\n▸ Enclosures, Racks, Cabinets & Panel Products\n▸ Events\n▸ Interconnection\n▸ Manufacturi..."
1,http://www.mysmartrend.com/news-briefs/technical-analysis/cr-bard-inc-has-returned-489-smartrend-recommendation-bcr-2,2020-04-17,en,Cr Bard Inc Has Returned 48.9% Since SmarTrend Recommendation (BCR) | Comtex SmarTrend,"\n\nCr Bard Inc Has Returned 48.9% Since SmarTrend Recommendation (BCR) | Comtex SmarTrend\n \nOWL LOGIN / \n ADVISOR LOGIN\n\nHome\nAdvantages\nProducts\n\nNews &amp Analysis\n\n\nResources\nTestimonials\n\n *Supporting investors at home - 60% off SmarTrend Products - Coupon Code: COVID19 \nReturn to Headlines\nCr Bard Inc Has Returned 48.9% Since SmarTrend Recommendation (BCR) \nWritten on Fri, 04/17/2020 - 1:08am\n By Shiri Gupta\nSmarTrend identified an Uptrend for Cr Ba..."
2,https://abcnews.go.com/Entertainment/video/ai-hiring-firing-focus-book-journalist-hilke-schellmann-108912618,2024-04-06,en,Video AI used for hiring and firing focus of book by journalist Hilke Schellmann - ABC News,\n\nVideo AI used for hiring and firing focus of book by journalist Hilke Schellmann - ABC News\n\nABC NewsVideoLiveShowsElections538ShopInterest Successfully AddedWe'll notify you here with news aboutTurn on desktop notifications for breaking stories about interest? OffOnLOG INStream onLatest:Northeast earthquakeBiden in BaltimoreHow to photograph solar eclipseJobs report analysisAnti-DEI legislationRochester solar eclipseAI used for hiring and firing focus of book by journalist Hilke Schel...
3,https://boingboing.net/2020/08/14/cory-doctorow-experiments-with.html,2020-08-14,en,Cory Doctorow experiments with AI writing partner Sudowrite / Boing Boing,"\nCory Doctorow experiments with AI writing partner Sudowrite / Boing Boing\n /\n\tMark Frauenfelder /\n\t8:00 am Fri\n\n\t\t Aug 14, 2020 \t \nCory Doctorow experiments with AI writing partner Sudowrite\nFollow Us\nTwitter /\n\t\t\t\t Facebook /\n\t\t\t\t RSS\n Image: Depositphotos \nGPT-3 is a machine-learning trained language model that generates text based on a text prompt. Cory Doctorow has been playing around with a closed-beta implementation of Sudowrite, which he describes as a ""GPT3..."
4,https://cio.economictimes.indiatimes.com/news/next-gen-technologies/top-ai-ceos-experts-raise-risk-of-extinction-from-ai/100637242,2023-05-31,en,"Artificial Intelligence: Top AI CEOs, experts raise 'risk of extinction' from AI, ET CIO","\n\n\nArtificial Intelligence: Top AI CEOs, experts raise 'risk of extinction' from AI, ET CIO\n\n \n\nX\n\n\nWe use cookies to ensure best experience for you\nWe use cookies and other tracking technologies to improve your browsing experience on our site, show personalize content and targeted ads, analyze site traffic, and understand where our audience is coming from. You can also read our privacy policy, We use cookies to ensure the best experience for you on our website.\nBy choosing I acc..."


In [ ]:
# Converts the 'date' column to datetime format and extracts year, month, day, and day of the week into new columns
df = df_news_final_project.copy()
df["date"] = pd.to_datetime(df["date"])
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day
df["dayofweek"] = df["date"].dt.dayofweek

In [ ]:
# Cleans text data by removing URLs, special characters, emojis, and publication names
def clean_text(text):
    import nltk
    import re

    stopwords = set(nltk.corpus.stopwords.words('english'))
    lemmatizer = nltk.WordNetLemmatizer()
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', text)  # Remove URLs
    text = re.sub(r'www\S+', '', text)      # Remove URL headers
    text = re.sub(r'[\n\r\t]', ' ', text)    # Remove newlines, tab spaces
    text = re.sub(r'#\w+', '', text)     # Remove hashtags
    text = re.sub(r'@\w+', '', text)     # Remove mentions

    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

    text = emoji_pattern.sub(r'', text)     # Remove emojis

    # Removing all publication names
    text = re.sub(r'\b\w+\b\s+News\b', "", text)
    text = re.sub(r'\b\w+\b\s+Times\b', "", text)
    text = re.sub(r'\b\w+\b\s+Magazine\b', "", text)
    text = re.sub(r'\b\w+\b\s+Journal\b', "", text)
    text = re.sub(r'\b\w+\b\s+Post\b', "", text)
    text = re.sub(r'\b\w+\b\s+Mail\b', "", text)
    text = re.sub(r'\b\w+\b\s+Press\b', "", text)
    text = re.sub(r'\b\w+\b\s+Tribune\b', "", text)

    # Removing Share/Follow/Post
    text = re.sub(r'(Share|Post) (to|on) [\w\s]+', '', text)
    text = re.sub(r'Follow [\w\s]+ more', '', text)
    text = re.sub(r'Click [\w\s]+(here|to)[\w\s]+', '', text)
    text = re.sub(r'Read [\w\s]+ at [\w\s]+\.[a-z]+', '', text)

    text = re.sub(r'[^\w\s]', '', text)

    return text

In [ ]:
# Tokenizes text, removes single characters, numbers, stopwords, and performs lemmatization
def tokenize_text(text):

  stopwords = set(nltk.corpus.stopwords.words('english'))
  lemmatizer = nltk.WordNetLemmatizer()

  text = text.lower()
  words = text.split(" ")
  words = nltk.word_tokenize(text)

  # Remove single-character tokens (mostly punctuation)
  words = [word for word in words if len(word) > 1]

  # Remove numbers
  words = [word for word in words if not word.isnumeric()]

  # Remove stopwords
  words = [word for word in words if word not in stopwords]

  #Lemmatization
  words = [lemmatizer.lemmatize(word) for word in words]

  # return " ".join(words)
  return words

In [ ]:
# Applies the clean_text function in parallel to the 'title' and 'text' columns to create new cleaned columns
df['title_cleaned'] = df['title'].parallel_apply(clean_text)
df['text_cleaned'] = df['text'].parallel_apply(clean_text)

In [ ]:
# Calculates the length of cleaned titles and texts and stores them in new columns
df['title_length'] = df['title_cleaned'].parallel_apply(lambda x: len(x.split()))
df['text_length'] = df['text_cleaned'].parallel_apply(lambda x: len(x.split()))

In [ ]:
df['title_length'].describe(percentiles=[0.05, 0.95])

,title_length
count,199707.000000
mean,13.150656
std,6.888567
min,1.000000
5%,7.000000
50%,12.000000
95%,22.000000
max,1343.000000


In [ ]:
df['text_length'].describe(percentiles=[0.05, 0.95])

,text_length
count,199707.000000
mean,1169.371409
std,959.649689
min,3.000000
5%,132.000000
50%,984.000000
95%,2420.000000
max,28762.000000


In [ ]:
# Remove articles where title and text lengths are outside the 5th-95th percentile
df_cleaned = df[
    (df["title_length"] > 7)
    & (df["title_length"] < 22)
    & (df["text_length"] > 132)
    & (df["text_length"] < 2420)
]

In [ ]:
# Remove 'text_length' and 'title_length' columns
df_cleaned = df_cleaned.drop(columns=['text_length', 'title_length'])

In [ ]:
df_cleaned.shape

(159769, 11)

In [ ]:
# Tokenizes the cleaned title and text columns
df_cleaned['title_cleaned_tokens'] = df_cleaned['title_cleaned'].parallel_apply(tokenize_text)
df_cleaned['text_cleaned_tokens'] = df_cleaned['text_cleaned'].parallel_apply(tokenize_text)

In [ ]:
df_cleaned.head()

,url,date,language,title,text,year,month,day,dayofweek,title_cleaned,text_cleaned,title_cleaned_tokens,text_cleaned_tokens
0,http://www.dataweek.co.za/21690r,2024-04-05,en,Flash for AI - 28 March 2024 - EBV Electrolink - Dataweek,"\nFlash for AI - 28 March 2024 - EBV Electrolink - Dataweek\nHome\nAbout us\nBack issues / E-book / PDF\nSubscribe\nAdvertise\n\n \n\nEMP Handbook\n\nCategories\n\n▸ Editor's Choice\n▸ Multimedia, Videos\n▸ AI & ML\n▸ Analogue, Mixed Signal, LSI\n▸ Circuit & System Protection\n▸ Computer/Embedded Technology\n▸ Design Automation\n▸ DSP, Micros & Memory\n▸ Edge Computing & IIoT\n▸ Electronics Technology\n▸ Enclosures, Racks, Cabinets & Panel Products\n▸ Events\n▸ Interconnection\n▸ Manufacturi...",2024,4,5,4,Flash for AI 28 March 2024 EBV Electrolink Dataweek,Flash for AI 28 March 2024 EBV Electrolink Dataweek Home About us Back issues Ebook PDF Subscribe Advertise EMP Handbook Categories Editors Choice Multimedia Videos AI ML Analogue Mixed Signal LSI Circuit System Protection ComputerEmbedded Technology Design Automation DSP Micros Memory Edge Computing IIoT Electronics Technology Enclosures Racks Cabinets Panel Products Events Interconnection Manufacturing Production Technology Hardware Services News OptoElec...,"[flash, ai, march, ebv, electrolink, dataweek]","[flash, ai, march, ebv, electrolink, dataweek, home, u, back, issue, ebook, pdf, subscribe, advertise, emp, handbook, category, editor, choice, multimedia, video, ai, ml, analogue, mixed, signal, lsi, circuit, system, protection, computerembedded, technology, design, automation, dsp, micros, memory, edge, computing, iiot, electronics, technology, enclosure, rack, cabinet, panel, product, event, interconnection, manufacturing, production, technology, hardware, service, news, optoelectronics, ..."
1,http://www.mysmartrend.com/news-briefs/technical-analysis/cr-bard-inc-has-returned-489-smartrend-recommendation-bcr-2,2020-04-17,en,Cr Bard Inc Has Returned 48.9% Since SmarTrend Recommendation (BCR) | Comtex SmarTrend,"\n\nCr Bard Inc Has Returned 48.9% Since SmarTrend Recommendation (BCR) | Comtex SmarTrend\n \nOWL LOGIN / \n ADVISOR LOGIN\n\nHome\nAdvantages\nProducts\n\nNews &amp Analysis\n\n\nResources\nTestimonials\n\n *Supporting investors at home - 60% off SmarTrend Products - Coupon Code: COVID19 \nReturn to Headlines\nCr Bard Inc Has Returned 48.9% Since SmarTrend Recommendation (BCR) \nWritten on Fri, 04/17/2020 - 1:08am\n By Shiri Gupta\nSmarTrend identified an Uptrend for Cr Ba...",2020,4,17,4,Cr Bard Inc Has Returned 489 Since SmarTrend Recommendation BCR Comtex SmarTrend,Cr Bard Inc Has Returned 489 Since SmarTrend Recommendation BCR Comtex SmarTrend OWL LOGIN ADVISOR LOGIN Home Advantages amp Analysis Resources Testimonials Supporting investors at home 60 off SmarTrend Products Coupon Code COVID19 Return to Headlines Cr Bard Inc Has Returned 489 Since SmarTrend Recommendation BCR Written on Fri 04172020 108am By Shiri Gupta SmarTrend identified an Uptrend for Cr Bard Inc BCR on December 23rd 2016 at 22245 In approximate...,"[cr, bard, inc, returned, since, smartrend, recommendation, bcr, comtex, smartrend]","[cr, bard, inc, returned, since, smartrend, recommendation, bcr, comtex, smartrend, owl, login, advisor, login, home, advantage, amp, analysis, resource, testimonial, supporting, investor, home, smartrend, product, coupon, code, covid19, return, headline, cr, bard, inc, returned, since, smartrend, recommendation, bcr, written, fri, 108am, shiri, gupta, smartrend, identified, uptrend, cr, bard, inc, bcr, december, 23rd, approximately, month, cr, bard, inc, returned, today, recent, price, 3312..."
2,https://abcnews.go.com/Entertainment/video/ai-hiring-firing-focus-book-journalist-hilke-schellmann-108912618,2024-04-06,en,Video AI used for hiring and firing focus of book by journalist Hilke Schellmann - ABC News,\n\nVideo AI used for hiring and firing focus of book by journalist Hilke Schellmann - ABC News\n\nABC NewsVideoLiveShowsEl

In [ ]:
print('Minimum date:', df_cleaned['date'].min())

Minimum date: 2020-01-01 00:00:00


In [ ]:
print('Maximum date:', df_cleaned['date'].max())

Maximum date: 2025-02-11 00:00:00


In [ ]:
# Saves the cleaned and tokenized dataframe to a parquet file in Google Drive
df_cleaned.to_parquet('/content/drive/My Drive/df_cleaned_tokenized.parquet')